# MLE para anisotropía geométrica

Recordemos los supuestos realizados,
$$\mathbf{y} = (y_1, \ldots, y_n)^\top \sim \text{N}(0,\mathbf{\Sigma}(\mathbf{A}))$$

$$\mathbf{\Sigma}(\mathbf{A})_{i,j} = c(\mathbf{s_i},\mathbf{s_j},\mathbf{A}) = C(\mathbf{h}:=\mathbf{s_i}-\mathbf{s_j}, \mathbf{A}) = \mathcal{M}_{\theta,\nu = 3/2}\left(\sqrt{\mathbf{h}^\top \mathbf{A} \mathbf{h}}\right)$$

$$\mathcal{M}(x)_{\theta,\nu = 3/2} = (1+x/\theta) \exp\left( -x/\theta \right), \qquad \mathbf{A} = \mathbf{P}(\alpha)\mathbf{D}(\text{ratio})\mathbf{P}^\top(\alpha)$$
donde
$$\mathbf{P}(\alpha) = \begin{pmatrix}\cos{\alpha} & \sin{\alpha} \\ -\sin{\alpha} & \cos{\alpha}\end{pmatrix}, \qquad \mathbf{D}(\text{ratio}) = \begin{pmatrix}1 & 0 \\ 0 &\text{ratio}\end{pmatrix},\qquad \alpha\in[0,\pi), \theta>0, \text{ratio}\in(0,1]$$

Luego, la función de log-verosimilitud para la matriz de parámetros $\mathbf{A}$ y el campo espacial $\mathbf{y}$ viene dada por:
$$\ell(\mathbf{A}, \mathbf{y}) = -\frac{1}{2}\left[\mathbf{y}^\top\mathbf{\Sigma^{-1}(\mathbf{A})}\mathbf{y} + \log\det\left(\mathbf{\Sigma(\mathbf{A})}\right) + n\log(2\pi)\right]$$

Similar al paper, para que los resultados sean comparables se calculará el estimador máximo verosimil de los parámetros, evaluando en cada configuración creada del conjunto de entrenamiento utilizada para entrenar la red neuronal.

Importamos las librerías necesarias.

In [6]:
import numpy as np
from scipy.linalg import cholesky as chol, cho_solve

from joblib import Parallel, delayed
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

Definimos algunas funciones auxiliares.

In [7]:
def generar_grilla(sqrt_n):
    xx = np.linspace(1,sqrt_n,sqrt_n)
    X, Y = np.meshgrid(xx,xx)
    return np.column_stack((X.flatten(), Y.flatten())) #Ordenados de izq a der y de abajo hacia arriba

P = lambda alpha: np.array([[np.cos(alpha), np.sin(alpha)],[-np.sin(alpha), np.cos(alpha)]])
D = lambda ratio: np.diag([1,ratio])
A = lambda alpha, ratio: P(alpha) @ D(ratio) @ P(alpha).T

matern_model = lambda t, x: (1 + x/t) * np.exp(-x/t) # nu=3/2

def cholesky(alpha, t, ratio, H):
    sqrt_H_TAH = np.sqrt(np.einsum('ijk,ijk->ij', H @ A(alpha, ratio) , H))
    sigma = matern_model(t, sqrt_H_TAH)
    return chol(sigma, lower=True, overwrite_a=True) # np.linalg.cholesky(sigma)

def ML(alpha, t, ratio, H, Z):
    '''
    Z debe ser un vector con el orden correcto dado por los puntos.
    '''
    L = cholesky(alpha, t, ratio, H)
    log_det_cov = 2 * np.sum(np.log(np.diag(L)))
    temp = cho_solve((L, True), Z)
    return -0.5 * (log_det_cov + np.dot(Z, temp))

def maximize(Z, y_train, H):
    '''
    Z debe ser un vector con el orden correcto dado por los puntos.
    '''
    max_value = -np.inf
    for alpha, t, ratio in y_train:
        if ML(alpha, t, ratio, H, Z) > max_value:
            max_value = ML(alpha, t, ratio, H, Z)
            MLE = (alpha, t, ratio)
    return MLE

def maximizar(Z, y_train, H): # Version paralelizada
    results = Parallel(n_jobs=12)(delayed(ML)(alpha, theta, ratio, H, Z) for alpha, theta, ratio in y_train)
    return y_train[results.index(max(results))]

Generamos el tensor de distancias

In [8]:
sqrt_n = 16
points = generar_grilla(sqrt_n)
H = points[:, np.newaxis, :] - points[np.newaxis, :, :]

Importamos el conjunto de entrenamiento y prueba generado en el primer script.

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
path = "/content/drive/MyDrive/10sem 2023-2/Ayud Inv/data/"

In [11]:
X_test = np.load(path + 'X_test.npy')

# Calculamos MLE optimizando

In [12]:
from scipy.optimize import minimize

def negative_log_likelihood(params, H, Z):
    alpha, t, ratio = params
    return -ML(alpha, t, ratio, H, Z)

In [13]:
initial_guess = [np.deg2rad((0+180)/2), (0.02+5)/2, (0+1)/2]
tol = 1e-6
bounds = [(0, np.pi-tol), (0.0+tol, np.inf), (0.0+tol, 1.0)]
y_pred_ML_opt = []
X_test.shape

(54000, 16, 16)

In [14]:
%%time
for field in tqdm(X_test, desc='progress', ncols=100, leave=False):
    Z = np.flipud(field).ravel()
    y_pred_ML_opt.append(minimize(negative_log_likelihood, initial_guess, args=(H, Z), bounds=bounds, method='L-BFGS-B').x)

CPU times: user 4h 22min 39s, sys: 3h 8min 7s, total: 7h 30min 47s
Wall time: 4h 32min 9s


In [15]:
np.save(path+'y_pred_ML_opt.npy', np.array(y_pred_ML_opt))